<script src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js"></script>

<div style="text-align: left;">
    <h1>Webscraping</h1>
    <h4>Applications of Cloud Computing and Big Data - ECON 446</h3>
    <div style="padding: 20px 0;">
        <hr style="border: 0; height: 1px; background-image: linear-gradient(to right, rgba(0, 0, 0, 0), rgba(0, 0, 0, 0.75), rgba(0, 0, 0, 0));">
        <p><em>Bella Rakhlina</em><br>
        <em>Lora Yovcheva</em><br>
        <em>Mauricio Vargas-Estrada</em><br>
        <br>Master Of Quantitative Economics<br>
        University of California - Los Angeles</p>
        <hr style="border: 0; height: 1px; background-image: linear-gradient(to right, rgba(0, 0, 0, 0), rgba(0, 0, 0, 0.75), rgba(0, 0, 0, 0));">
    </div>
</div>

## Web Crawling Tables

<div style="border: 1px solid black; border-radius: 5px; overflow: hidden;">
    <div style="background-color: black; color: white; padding: 5px; text-align: left;">
       a.) 
    </div>
    <div style="padding: 10px;">
        Create a list of links for all the wikipedia pages for NYSE traded companies A-Z and 0-9.
</div>

In [1]:
import requests
import string
import re
import toolz
import time
import pickle
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from multiprocessing import Pool
from tqdm import tqdm
from rich import inspect
from pprint import pprint

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
headers = {
    'Accept-Encoding': 'gzip, deflate, br',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
}

In [7]:
url = lambda x: f'https://en.wikipedia.org/wiki/Companies_listed_on_the_New_York_Stock_Exchange_({x})'

In [8]:
def get_table_info(soup):
    links = []
    names = []
    tickers = []

    # For each row in the table of companies.
    for r in soup.find_all('tr'):
        # Get the columns for the row `r`.
        row = r.find_all('td')
        # Check if the row contains data.
        if row:
            # Extract the link of the company in the row `r`.
            tmp_link = row[0].find_all('a')
            # Check if the company has a link.
            if tmp_link:
                names.append(tmp_link[0].text)
                # Adding the root to the relative path.
                links.append('https://www.wikipedia.org' + tmp_link[0].get('href'))
                tickers.append(row[1].text.strip())
    return names, tickers, links        

In [ ]:
links = []
names = []
tickers = []
for letter in tqdm(list(string.ascii_uppercase) + ['0-9']):
    req = requests.get(url(letter), headers=headers, timeout=20)
    # Pass if the request is not successful or time out.
    if not req.ok:
        continue
    soup = BeautifulSoup(req.content, 'html.parser')
    tmp_n, tmp_t, tmp_l= get_table_info(soup)
    links.extend(tmp_l)
    names.extend(tmp_n)
    tickers.extend(tmp_t)

In [ ]:
companies = pd.DataFrame({'company': names, 'ticker': tickers, 'link': links})

In [10]:
#companies.to_csv('data/companies_links.csv', index=False)
#companies = pd.read_csv('data/companies_links.csv')
companies = companies.drop_duplicates()

In [11]:
print(companies.head())

                   company ticker  \
0  A. O. Smith Corporation    AOS   
1       A10 Networks, Inc.   ATEN   
2          AAR Corporation    AIR   
3             Aaron's Inc.    AAN   
4                 ABB LTD.    ABB   

                                             link  
0      https://www.wikipedia.org/wiki/A._O._Smith  
1     https://www.wikipedia.org/wiki/A10_Networks  
2         https://www.wikipedia.org/wiki/AAR_Corp  
3  https://www.wikipedia.org/wiki/Aaron%27s,_Inc.  
4        https://www.wikipedia.org/wiki/ABB_Group  


<div style="border: 1px solid black; border-radius: 5px; overflow: hidden;">
    <div style="background-color: black; color: white; padding: 5px; text-align: left;">
       b.) 
    </div>
    <div style="padding: 10px;">
        Crawl through all the URLs and make 1 DF with all the NYSE publically traded companies.
</div>

In [ ]:
def get_type(url):
    headers = {
        'Accept-Encoding': 'gzip, deflate, br',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
    }
    # Crawling the wiki page of the company. Rise and exception if the url
    # is broken.
    try:
        req = requests.get(url, headers=headers, timeout=20)
    except:
        print(f'Request failed in {url}')
        return ''
    # Pass if the request is not successful or time out.
    if not req.ok:
        return ''
    soup = BeautifulSoup(req.content, 'html.parser')
    # Extracting the type of the company from the infobox vcard. Rise an
    
    try:
        vcard = soup.find_all('table', {'class': 'infobox vcard'})[0]
        comp_type = vcard.find('a', {'title': 'Public company'}).text
    except:
        print(f'No company type in {url}')
        return ''
    return comp_type

In [ ]:
# Multiprocessing to crawl the websites in parallel. 
with Pool(100) as p:
    comp_type = p.map(get_type, companies['link'])

In [ ]:
# Type of companies in lowercase.
companies['type'] = [x.lower() for x in comp_type]

In [ ]:
# Because the format is not uniform between wikis, extract the word 'public'
# from those companies that have that pattern in their type.
pattern = re.compile(r'.*public.*')
companies['type'] = [
    pattern.sub('public', s) 
    if pattern.match(s) else s for s in companies.type
]

In [12]:
#companies.to_csv('data/companies_links_with_type.csv', index=False)
#companies = pd.read_csv('data/companies_links_with_type.csv')

In [13]:
# Companies that are public according to their wiki page.
print(companies.query("type=='public'"))

                           company ticker  \
0          A. O. Smith Corporation    AOS   
1               A10 Networks, Inc.   ATEN   
2                  AAR Corporation    AIR   
3                     Aaron's Inc.    AAN   
4                         ABB LTD.    ABB   
...                            ...    ...   
1522  Zimmer Biomet Holdings, Inc.    ZBH   
1524                   Zoetis Inc.    ZTS   
1525                   Zuora, Inc.    ZUO   
1526        3D Systems Corporation    DDD   
1527                    3M Company    MMM   

                                                link    type  
0         https://www.wikipedia.org/wiki/A._O._Smith  public  
1        https://www.wikipedia.org/wiki/A10_Networks  public  
2            https://www.wikipedia.org/wiki/AAR_Corp  public  
3     https://www.wikipedia.org/wiki/Aaron%27s,_Inc.  public  
4           https://www.wikipedia.org/wiki/ABB_Group  public  
...                                              ...     ...  
1522    https://ww

<div style="border: 1px solid black; border-radius: 5px; overflow: hidden;">
    <div style="background-color: black; color: white; padding: 5px; text-align: left;">
       c.) 
    </div>
    <div style="padding: 10px;">
        What is the percetages of companies that contain 1 letter, 2 letters, 3 letters, 4 letters, 5 letters,... in the ticker (drop punctuation)?
</div>

In [14]:
# Cleaning the ticker column.
companies['clean_ticker'] = [
    re.match(r"[^,.!? ]+", x).group(0)
    for x in companies.ticker
]

In [15]:
# Calculating the length of the ticker.
companies['len_ticker'] = [len(x) for x in companies.clean_ticker]

In [16]:
# Counting the percentage of companies with each length of ticker.
len_ticker_pct = toolz.pipe(
    companies[['clean_ticker', 'len_ticker']],
    lambda x: x.drop_duplicates(),
    lambda x: x.groupby('len_ticker'),
    lambda x: x.count(),
    lambda x: x * 100 / x.sum()
)

In [17]:
for i, row in len_ticker_pct.iterrows():
    tmp = f'''
    -----------------------------------------
        Length of the ticker: {i}
        % in the total of companies: {row.clean_ticker:0.2f}
    -----------------------------------------
    '''
    print(tmp)


    -----------------------------------------
        Length of the ticker: 1
        % in the total of companies: 1.52
    -----------------------------------------
    

    -----------------------------------------
        Length of the ticker: 2
        % in the total of companies: 10.13
    -----------------------------------------
    

    -----------------------------------------
        Length of the ticker: 3
        % in the total of companies: 72.58
    -----------------------------------------
    

    -----------------------------------------
        Length of the ticker: 4
        % in the total of companies: 15.76
    -----------------------------------------
    


## Web Scraping Using Beautiful Soup

<div style="border: 1px solid black; border-radius: 5px; overflow: hidden;">
    <div style="background-color: black; color: white; padding: 5px; text-align: left;">
       a.) 
    </div>
    <div style="padding: 10px;">
        Using Beautiful soup .findAll method you will webscrape the front page of Reddit. Get a list of all of the "timestamps"
</div>

In [3]:
# User agent to simulate a browser in selenium.
options = Options()
options.set_preference("general.useragent.override", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")

In [ ]:
# Initialize the browser.
driver = webdriver.Firefox(options=options)

In [ ]:
# Open the website in the browser.
driver.get("https://www.reddit.com")
time.sleep(5)

# Verifying the numbers of posts extracted from the page.
num_posts_before = 0

# posts inside the website to scroll down.
body = driver.find_element(By.TAG_NAME, 'body')

start_time = time.time()
time_limit_scroll = 600 # 10 minutes

# Scrolling down the page until the time limit is reached.
print('Scrolling down the page...')
while time.time() - start_time < time_limit_scroll:
    print(f'Time elapsed: {time.time() - start_time:.2f} seconds')
    # Extracting the articles from the page.
    posts = driver.find_elements(By.XPATH, '//article[@class="w-full m-0"]')
    
    # Verifying that the number of posts is changing in each iteration.
    if len(posts) == num_posts_before:
        print('Something went wrong. The number of posts is not changing. Stopping..')
        break  
    else:
        num_posts_before = len(posts)

    # Simulate pressing the END key to scroll down the page.
    body.send_keys(Keys.END)

    # Wait for the page to load.
    time.sleep(6)
print('Time limit reached.')

In [ ]:
# Extracting the html for each post.
html_content = [x.get_attribute('outerHTML') for x in posts]

In [ ]:
# Save the html content to a pickle file.
with open('data/reddit_posts.pkl', 'wb') as f:
    pickle.dump(html_content, f)

In [ ]:
# Close the browser.
driver.quit()

In [2]:
# Load the html content from the pickle file to avoid running the code again.
#with open('data/reddit_posts.pkl', 'rb') as f:
    #html_content = pickle.load(f)

In [3]:
# Parsing each post with BeautifulSoup.
html_content = [BeautifulSoup(x, 'html.parser') for  x in html_content]

In [4]:
# Extracting only the timestamps for each post.
timestamps = [x.find('time').get_attribute_list('datetime')[0] for x in html_content]

In [5]:
print(f'Total number of posts extracted: {len(timestamps)}')
print('First 10 timestamps:')
pprint(timestamps[0:9])

Total number of posts extracted: 2478
First 10 timestamps:
['2024-04-19T07:10:50.758Z',
 '2024-04-19T17:25:09.146Z',
 '2024-04-19T13:10:37.611Z',
 '2024-04-19T11:20:55.005Z',
 '2024-04-19T15:02:59.261Z',
 '2024-04-19T19:17:01.403Z',
 '2024-04-19T02:00:27.276Z',
 '2024-04-19T11:38:59.637Z',
 '2024-04-19T19:45:27.032Z']


<div style="border: 1px solid black; border-radius: 5px; overflow: hidden;">
    <div style="background-color: black; color: white; padding: 5px; text-align: left;">
       b.) 
    </div>
    <div style="padding: 10px;">
        Using the functions findChild, descendents, etc. locate the post title, text and post time into a dataframe.
</div>

In [6]:
titles = [x.find('article').get_attribute_list('aria-label')[0] for x in html_content]

In [7]:
text_div_class = re.compile(r'.*feed-card-text-preview.*')
texts = []
for post in html_content:
    tmp = post.find_all(
        'div',
        class_ = lambda x : x and text_div_class.match(x)    
    )
    if tmp:
        tmp = tmp[0].text
    else:
        tmp = np.nan
    texts.append(tmp)

In [8]:
post_df = pd.DataFrame({
    'timestamp': timestamps,
    'title': titles,
    'text': texts
})

In [12]:
# Printing the first 10 posts with text.
print('First 5 posts with text:')
print(post_df.query('text.notna()').head(5))

First 5 posts with text:
                   timestamp  \
2   2024-04-19T13:10:37.611Z   
21  2024-04-19T15:55:01.669Z   
23  2024-04-19T16:35:15.247Z   
28  2024-04-19T05:30:53.159Z   
34  2024-04-19T15:22:26.286Z   

                                                title  \
2   My husband won't let me take more than two sho...   
21  AITAH for breaking up with my bf after he alle...   
23  After years of tipping 20-25% I’m DONE. I’m ti...   
28  Orbital cooldowns are way too long compared to...   
34  Is it just me or is the new Taylor swift album...   

                                                 text  
2   \n\n    This is the weirdest thing my husband ...  
21  \n\n    I’m not a clubbing kind of girl. My bf...  
23  \n\n    I have always “over tipped” according ...  
28  \n\n    Orbital Precision Strike has a 100s co...  
34  \n\n    I'm not here to be a hater but I felt ...  


## RegEx

<div style="border: 1px solid black; border-radius: 5px; overflow: hidden;">
    <div style="background-color: black; color: white; padding: 5px; text-align: left;">
       a.) 
    </div>
    <div style="padding: 10px;">
        Using RegEx, get all the urls of ladder faculty profiles for UCLA Economics
</div>

In [4]:
URL = "https://economics.ucla.edu/faculty/ladder"

In [ ]:
# Getting the flex table of the ladder faculty.
req_ladder = requests.get(URL, headers=headers)

In [ ]:
# Extracting the name and url for each profile in the ladder faculty.
tmp = re.compile(r'flex_column av_one_fourth.*')
profiles_info = toolz.pipe(
    req_ladder,
    lambda x: x.content,
    lambda x: BeautifulSoup(x, 'html.parser'),
    lambda x : x.find(
        'div',
        {'id': 'wpv-view-layout-974-CATTR0494cfbfb8d3e1b3152203680573333f'}
    ),
    lambda x: x.find_all(
        'div',
        class_ = lambda x : x and tmp.match(x)
    ),
    lambda x: [y.find('h3') for y in x],
    lambda x: {
        'name': [y.text for y in x],
        'url': [y.find('a')['href'] for y in x]
    },
    lambda x: pd.DataFrame(x)
)

<div style="border: 1px solid black; border-radius: 5px; overflow: hidden;">
    <div style="background-color: black; color: white; padding: 5px; text-align: left;">
       b.) 
    </div>
    <div style="padding: 10px;">
        Webcrawl the links from A and use RegEx to get all the emails and phone numbers of ladder faculty profiles.
</div>

In [ ]:
# Crawling the profiles urls.
crawled_info = []
for url in tqdm(profiles_info['url']):
    tmp = requests.get(url, headers=headers).content
    crawled_info.append(tmp)
    time.sleep(10)

In [ ]:
# Extracting the email and phone from the crawled info using regex.
regex_email = re.compile(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+')
regex_phone = re.compile(r'([\+]?[(]?[0-9]{3}[)]?[-\s\.][0-9]{3}[-\s\.][0-9]{4,6})')
email = []
phone = []
for raw in crawled_info:
    tmp = BeautifulSoup(raw, 'html.parser')
    tmp_email = tmp.find(string=regex_email)
    tmp_phone = tmp.find(string=regex_phone)

    if tmp_email:
        email.append(tmp_email)
    else:
        email.append(np.nan)
    
    if tmp_phone:
        phone.append(tmp_phone)
    else:
        phone.append(np.nan)

In [ ]:
profiles_info['email'] = email
profiles_info['phone'] = phone

In [14]:
# Printing those profiles with phone number.
print(profiles_info.query('phone.notna()').head(10))

                    name                                                url  \
8         Ariel Burstein  https://economics.ucla.edu/person/ariel-burstein/   
12      Pablo Fajgelbaum  https://economics.ucla.edu/person/pablo-fajgel...   
17    Martin B. Hackmann  https://economics.ucla.edu/person/martin-b-hac...   
18          Jinyong Hahn    https://economics.ucla.edu/person/jinyong-hahn/   
20        Hugo Hopenhayn  https://economics.ucla.edu/person/hugo-hopenhayn/   
23  Adriana Lleras-Muney  https://economics.ucla.edu/person/adriana-ller...   
25                Jay Lu          https://economics.ucla.edu/person/jay-lu/   
26          Rosa Matzkin  https://economics.ucla.edu/person/rosa-liliana...   
27     Maurizio Mazzocco  https://economics.ucla.edu/person/maurizio-maz...   
28      Kathleen McGarry  https://economics.ucla.edu/person/kathleen-mcg...   

                        email            phone  
8        arielb@econ.ucla.edu   (310) 206-6732  
12  pfajgelbaum@econ.ucla.edu   

In [7]:
tmp = f"""
Number of profiles: {profiles_info.shape[0]}
Number of profiles with phone number: {profiles_info.query('phone.notna()').shape[0]}
"""
print(tmp)


Number of profiles: 45
Number of profiles with phone number: 16



## Selenium

<div style="border: 1px solid black; border-radius: 5px; overflow: hidden;">
    <div style="background-color: black; color: white; padding: 5px; text-align: left;">
       a.) 
    </div>
    <div style="padding: 10px;">
        Pick a website that has useful data to a business or economic question. Put your website you plan to scrape here : 
        
        https://docs.google.com/spreadsheets/d/1PJ2DOTCVCh51fn0ry1yB7qTyccR33_IXFpkYdd58MFs/edit?usp=sharing

        You must have use website that no other group has. First come first serve.
</div>

<div style="border: 1px solid black; border-radius: 5px; overflow: hidden;">
    <div style="background-color: black; color: white; padding: 5px; text-align: left;">
       b.) 
    </div>
    <div style="padding: 10px;">
        Use Selenium to scrape valuable information from your website and store in a dataframe.
</div>

<div style="border: 1px solid black; border-radius: 5px; overflow: hidden;">
    <div style="background-color: black; color: white; padding: 5px; text-align: left;">
       c.) 
    </div>
    <div style="padding: 10px;">
        Write a short paragraph about the businesses or research that would use the data you scraped. Describe it's value and what it can be used for.
</div>